In [24]:
!git clone https://github.com/valmirf/redes_neurais_pos.git

fatal: destination path 'redes_neurais_pos' already exists and is not an empty directory.


##Multipayer Perceptron (MLP)

Rede Neural baseado no algoritmo de gradiente descendente.  
Os gradientes são calculados usando backpropagation.

Para mais detalhes, ver os capitulos 13 a 16 do livro no site:

http://deeplearningbook.com.br/

In [25]:
import random
import numpy as np

A entrada é uma lista (`sizes`) contém o número de neurônios nas respectivas camadas da rede. Por exemplo, se a lista for [2, 3, 1] então será uma rede de três camadas, com o primeira camada contendo 2 neurônios, a segunda camada 3 neurônios, e a terceira camada 1 neurônio. Os bias e pesos para a rede são inicializados aleatoriamente, usando uma distribuição Gaussiana com média 0 e variância 1. Note que a primeira camada é assumida como uma camada de entrada, e por convenção não definimos nenhum bias para esses neurônios, pois os bias são usados na computação das saídas das camadas posteriores.


In [26]:
# Classe Network
class Network(object):

    def __init__(self, sizes):
        self.num_layers = len(sizes)  #número de neurônios em cada camada
        self.sizes = sizes

        #Inicialização dos pesos de forma mais adequada para o ReLu
        self.biases = [np.random.randn(y, 1) * np.sqrt(2.0/x) * 0.5 for x, y in zip(sizes[:-1], sizes[1:])]
        self.weights = [np.random.randn(y, x) * np.sqrt(2.0/x) * 0.5 for x, y in zip(sizes[:-1], sizes[1:])]

        #Inicialização dos pesos padrão
        # self.biases = [np.random.randn(y, 1) for y in sizes[1:]] #limiar
        # self.weights = [np.random.randn(y, x) for x, y in zip(sizes[:-1], sizes[1:])] #pesos
        
        # Inicialização do momentum
        self.momentum_w = [np.zeros(w.shape) for w in self.weights]
        self.momentum_b = [np.zeros(b.shape) for b in self.biases]

    def batch_normalize(self, x):
      mean = np.mean(x, axis=0)
      std = np.std(x, axis=0)
      normalized_data = (x - mean) / (std + 1e-8)
      return normalized_data

    def feedforward(self, x):
        """Retorna a saída da rede z se `x` for entrada."""
        for b, w in zip(self.biases, self.weights):
            x = self.batch_normalize(x)
            #x = sigmoid(np.dot(w, x)+b) #net = (∑xw+b)
            x = relu(np.dot(w, x)+b) #net = (∑xw+b)
        return x


    #def SGD(self, training_data, epochs, mini_batch_size, initial_eta, 𝜂, test_data=None):        
    def SGD(self, training_data, epochs, mini_batch_size, initial_eta, lambda_reg=0.0, momentum=0.0, test_data=None):
        """Treinar a rede neural usando o algoritmo mini batch com gradiente descendente.
         A entrada é uma lista de tuplas
         `(x, y)` representando as entradas de treinamento e as
         saídas. Os outros parâmetros não opcionais são
         auto-explicativos. Se `test_data` for fornecido, então a
         rede será avaliada em relação aos dados do teste após cada
         época e progresso parcial impresso. Isso é útil para
         acompanhar o progresso, mas retarda as coisas substancialmente."""
         
        #dataset de treino
        training_data = list(training_data)
        n = len(training_data)
        
        #dataset de teste
        if test_data:
            test_data = list(test_data)
            n_test = len(test_data)

        accuracies = []

        for j in range(epochs):
            eta = initial_eta * (0.95 ** j)
            random.shuffle(training_data)
            #técnica que realiza o treinamento por lotes
            #mini_batch_size = tamanho do lote
            mini_batches = [training_data[k:k + mini_batch_size] for k in range(0, n, mini_batch_size)]

            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, eta, lambda_reg, momentum)

            if test_data:
                acc = self.evaluate(test_data)
                accuracies.append((acc * 100) / n_test)
                print("Epoch {} : {} / {} = {:.2f}%".format(j, acc, n_test, (acc * 100) / n_test))
            else:
                print("Epoch {} finalizada".format(j))

        if test_data:
            avg_accuracy = np.mean(accuracies)
            print("\nAcurácia média após {} épocas: {:.2f}%".format(epochs, avg_accuracy))
    
    
    # def update_mini_batch(self, mini_batch, 𝜂):
    def update_mini_batch(self, mini_batch, eta, lambda_reg, momentum):
        """Atualiza os pesos e limiares da rede aplicando
         a descida do gradiente usando backpropagation para um único mini lote.
         O `mini_batch` é uma lista de tuplas `(x, y)`, e `a` é a taxa de aprendizado."""
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        nabla_b = [np.zeros(b.shape) for b in self.biases]

        for x, y in mini_batch:
            #resultado dos deltas do backpropagation sem a multiplicação da taxa de aprendizagem
            #soma os deltas do minibatch
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)
            nabla_w = [nw + dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
            nabla_b = [nb + dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]

        # atualizando os pesos e vieses com regularização L2
        self.weights = [(1 - eta * lambda_reg / len(mini_batch)) * w - (eta / len(mini_batch)) * nw + momentum * mw
                        for w, nw, mw in zip(self.weights, nabla_w, self.momentum_w)]
        self.biases = [b - (eta / len(mini_batch)) * nb + momentum * mb
                       for b, nb, mb in zip(self.biases, nabla_b, self.momentum_b)]

        # atualizando o vetor de momentum
        self.momentum_w = [momentum * mw + (eta / len(mini_batch)) * nw for mw, nw in zip(self.momentum_w, nabla_w )]
        self.momentum_b = [momentum * mb + (eta / len(mini_batch)) * nb for mb, nb in zip(self.momentum_b, nabla_b)]
    
    

    def backprop(self, x, y):
        """Retorna uma tupla `(nabla_b, nabla_w)` representando o
         gradiente para a função de custo J_x. `nabla_b` e
         `nabla_w` são listas de camadas de matrizes numpy, semelhantes
         a `self.biases` e `self.weights`."""
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        nabla_b = [np.zeros(b.shape) for b in self.biases]

        # Feedforward
        activation = x

        # Lista para armazenar todas as saídas dos neurônios (z), camada por camada
        activations = [x]

        # Lista para armazenar todos os vetores net, camada por camada
        nets = []

        for b, w in zip(self.biases, self.weights):
            net = np.dot(w, activation)+b
            nets.append(net)
            #activation = sigmoid(net) #z = valor de saída do neurônio
            activation = relu(net) #z = valor de saída do neurônio
            activations.append(activation)

        # Backward pass

        #última camada -(u-z)f'(net)
        #delta = self.cost_derivative(activations[-1], y) * sigmoid_prime(nets[-1]) #SIGMOID
        delta = self.cost_derivative(activations[-1], y) * relu_prime(nets[-1]) #RELU
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose()) #(𝑦−𝑧)*f’(net)*𝑥

        # l = 1 significa a última camada de neurônios, l = 2 é a penúltima e assim por diante.
        for l in range(2, self.num_layers):
            net = nets[-l]
            #zs = sigmoid_prime(net)
            zs = relu_prime(net)
            #delta da camada intermediaria. Note que utiliza o delta calculado anteriormente
            delta = np.dot(self.weights[-l+1].transpose(), delta) * zs
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose()) #∑(𝛿𝑤)f’(net)𝑥
        return (nabla_b, nabla_w)

    def evaluate(self, test_data):
        """Retorna o número de entradas de teste para as quais a rede neural
         produz o resultado correto. Note que a saída da rede neural
         é considerada o índice de qualquer que seja
         neurônio na camada final que tenha a maior ativação."""

        test_results = [(np.argmax(self.feedforward(x)), y) for (x, y) in test_data]
        return sum(int(x == y) for (x, y) in test_results)

    def cost_derivative(self, output_activations, y):
        """Retorna o vetor das derivadas parciais."""
        return (output_activations-y)

# Função de Ativação Sigmóide
def sigmoid(net):
    return 1.0/(1.0+np.exp(-net))

# Função para retornar as derivadas da função Sigmóide
def sigmoid_prime(z):
    return sigmoid(z)*(1-sigmoid(z))

# Função de Ativação ReLu
def relu(net):
    return np.maximum(0, net)

# Função para retornar as derivadas da função ReLu
def relu_prime(z):
    return np.where(z <= 0, 0, 1)


Como exemplo, essa mesma rede será executada na base de dados MNIST. O codigo abaixo carrega a base de dados.

In [27]:
# Carregar o dataset MNIST

# Imports
import pickle
import gzip
import numpy as np

def load_data():
    f = gzip.open('redes_neurais_pos/MLP/mnist.pkl.gz', 'rb')
    training_data, validation_data, test_data = pickle.load(f, encoding="latin1")
    f.close()
    return (training_data, validation_data, test_data)

def load_data_wrapper():
    tr_d, va_d, te_d = load_data()
    training_inputs = [np.reshape(x, (784, 1)) for x in tr_d[0]]
    training_results = [vectorized_result(y) for y in tr_d[1]]
    training_data = zip(training_inputs, training_results)
    validation_inputs = [np.reshape(x, (784, 1)) for x in va_d[0]]
    validation_data = zip(validation_inputs, va_d[1])
    test_inputs = [np.reshape(x, (784, 1)) for x in te_d[0]]
    test_data = zip(test_inputs, te_d[1])
    return (training_data, validation_data, test_data)

def vectorized_result(j):
    e = np.zeros((10, 1))
    e[j] = 1.0
    return e


#Executa a rede neural

Parâmetros de rede:
         2º param é contagem de épocas
         3º param é tamanho do lote
         4º param é a taxa de aprendizado (𝜂)




Verificando quais as melhores arquiteturas 

In [28]:
learning_rate = [0.1, 0.3, 0.5]

one_layer_arquitecture = [
  [784, 50, 10],
  [784, 64, 10],
  [784, 100, 10]
]

two_layer_arquitecture = [
  [784, 50, 25, 10],
  [784, 64, 32, 10],
  [784, 80, 40, 10]
]

architectures = [one_layer_arquitecture, two_layer_arquitecture]

for arq_grupo in architectures:
    print(f"Executando com grupo das seguintes configurações: 1 - {arq_grupo[0]} | 2 - {arq_grupo[1]} | 3 - {arq_grupo[2]}\n")
    for arq in arq_grupo:
        print(f"Configuração especifica a ser testada: {arq_grupo.index(arq)+1} - {arq}")
        print("----------------------------------------------------")
        for lr in learning_rate:
            training_data, validation_data, test_data = load_data_wrapper()
            training_data = list(training_data)

            mlp = Network(arq)
            print(f"Executando com taxa de aprendizado de {lr}")
            mlp.SGD(training_data, 10, 32, lr, test_data=test_data)
            print("\n")

print("Fim dos testes!")

DEFININDO AS ARQUITETURAS DOS 3 MELHORES RESULTADOS

melhores arquiteturas:

RELU - [784, 64, 10] - lr = 0.1 - 95.17%

RELU - [784, 100, 10] - lr = 0.1 - 94.16%

RELU - [784, 100, 10] - lr = 0.3 - 93.18%

In [29]:
training_data, validation_data, test_data = load_data_wrapper()
training_data = list(training_data)

#arquitetura da rede
#arquitecture = [784, 100, 10] #lr = 0.1 | lr = 0.3
arquitecture = [784, 64, 10] #lr = 0.1
mlp = Network(arquitecture)
# mlp.SGD(training_data, 30, 32, 0.1, test_data=test_data)
mlp.SGD(training_data, 15, 32, 0.1, lambda_reg=0.01, momentum=0.9, test_data=test_data)


Epoch 0 : 980 / 10000 = 9.80%
Epoch 1 : 980 / 10000 = 9.80%
Epoch 2 : 980 / 10000 = 9.80%


KeyboardInterrupt: 

Executando a arquitetura padrão do código fonte

In [ ]:
training_data, validation_data, test_data = load_data_wrapper()
training_data = list(training_data)

#arquitetura da rede
arquitecture = [784, 30, 10]
mlp = Network(arquitecture)
mlp.SGD(training_data, 30, 32, 0.3, test_data=test_data)




Epoch 0 : 4373 / 10000 = 43.73%
Epoch 1 : 7087 / 10000 = 70.87%
Epoch 2 : 7872 / 10000 = 78.72%
Epoch 3 : 8223 / 10000 = 82.23%
Epoch 4 : 8409 / 10000 = 84.09%
Epoch 5 : 8552 / 10000 = 85.52%
Epoch 6 : 8648 / 10000 = 86.48%
Epoch 7 : 8727 / 10000 = 87.27%
Epoch 8 : 8788 / 10000 = 87.88%
Epoch 9 : 8820 / 10000 = 88.2%
Epoch 10 : 8875 / 10000 = 88.75%
Epoch 11 : 8905 / 10000 = 89.05%
Epoch 12 : 8932 / 10000 = 89.32%
Epoch 13 : 8955 / 10000 = 89.55%
Epoch 14 : 8992 / 10000 = 89.92%
Epoch 15 : 8999 / 10000 = 89.99%
Epoch 16 : 9022 / 10000 = 90.22%
Epoch 17 : 9022 / 10000 = 90.22%
Epoch 18 : 9053 / 10000 = 90.53%
Epoch 19 : 9069 / 10000 = 90.69%
Epoch 20 : 9090 / 10000 = 90.9%
Epoch 21 : 9098 / 10000 = 90.98%
Epoch 22 : 9103 / 10000 = 91.03%
Epoch 23 : 9111 / 10000 = 91.11%
Epoch 24 : 9111 / 10000 = 91.11%
Epoch 25 : 9132 / 10000 = 91.32%
Epoch 26 : 9142 / 10000 = 91.42%
Epoch 27 : 9149 / 10000 = 91.49%
Epoch 28 : 9155 / 10000 = 91.55%
Epoch 29 : 9159 / 10000 = 91.59%


##**Mini-Projeto**

A partir dos melhores resultados obtidos no projeto anterior, execute 3 configurações pra cada questão a seguir:

1) Implementar as regularizações L1 e L2

2) Implementar o Momento

3) Comparar os experimentos e explicar o porque de cada resultado? Qual foi a melhor regularização? Por quê? O Momento melhorou os resultados? Por quê?

Data de Entrega: 14/10/2020
     
      
Complete a Tabela abaixo com os resultados

\begin{array}{|c|c|c|c|}\hline\\ \\
  Configuracao & 1 & 2 & 3 \\ \hline
L1 &  &  &   \\ \hline
L2  & & &  \\ \hline
Momento  & & & \\ \hline
\end{array}



\\
Data de Entrega: 17/10/2020
